In [1]:
import keras
from keras.layers import *
import numpy as np
import xml.etree.ElementTree as ET
import glob
from bs4 import BeautifulSoup
import os

In [2]:
encinput = keras.Input((1))
x = Dense(2048, activation="relu")(encinput)
x = Reshape((2, 1024))(x)
x = Conv1D(1024, 2, activation="relu")(x)
encoutput = Conv1D(1024,1, activation="relu")(x)

encoder = keras.Model(encinput, encoutput, name="encoder")
encoder.summary()

decinput = keras.Input((1, 1024))
x = Conv1D(1024,1, activation="relu")(decinput)
x = Conv1D(2048,1, activation="relu")(x)
x = Flatten()(x)
decoutput = Dense(1, activation="relu")(x)

decoder = keras.Model(decinput, decoutput, name="decoder")
decoder.summary()

autoencoder_input = keras.Input((1))
autoencoder_encoder = encoder(autoencoder_input)
autoencoder_decoder = decoder(autoencoder_encoder)

autoencoder = keras.Model(autoencoder_input, autoencoder_decoder, name="autoencoder")
autoencoder.summary()

autoencoder.compile(optimizer='adam', loss='accuracy')

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
dense (Dense)                (None, 2048)              4096      
_________________________________________________________________
reshape (Reshape)            (None, 2, 1024)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1, 1024)           2098176   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 1024)           1049600   
Total params: 3,151,872
Trainable params: 3,151,872
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
_________________________________________________________________
Layer (type)                 Out

In [12]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

data = []

for f in glob.glob(os.getcwd() + "/data/*.xml"):
    with open(f, errors='ignore') as fp:
        soup = BeautifulSoup(fp, 'xml')
        pageText = soup.findAll(text=True)
        pageText = "".join(pageText)
        pageText = chunks(pageText, 200)
        for i in pageText:
            data.append(i)

In [13]:
datachr = []
for i in data:
    row = []
    zeros = np.zeros((200))
    for j in i:
        row.append(ord(j))
    row = np.asarray(row).astype('float32').T
    zeros[:row.shape[0]] = row
    datachr.append(zeros)

In [14]:
sizes = []
for i in datachr:
    sizes.append(i.shape[0])
max(sizes)

200

In [ ]:
autoencoder.fit(datachr, datachr, epochs=1)